In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('https://www.tripadvisor.com/VacationRentalsBlog/2019/01/07/best-tropical-vacation-destinations/')
bs = BeautifulSoup(html, "html.parser")
titles = bs.find_all(['h3'])
print(*titles, sep='\n')

<h3>World’s Best Tropical Vacations</h3>
<h3>Bora Bora, French Polynesia</h3>
<h3>Port Douglas, Australia</h3>
<h3>Nassau, Bahamas</h3>
<h3>Tamarindo, Costa Rica</h3>
<h3>Kauai, Hawaii</h3>
<h3>Ambergris Caye, Belize</h3>
<h3>The Seychelles, East Africa</h3>
<h3>The Cook Islands, South Pacific</h3>
<h3>St. Kitts and Nevis, The Caribbean</h3>
<h3>Lombok, Indonesia</h3>
<h3>Ko Samui, Thailand</h3>
<h3>Tulum, Mexico</h3>
<h3>Langkawi, Malaysia</h3>
<h3>Bermuda</h3>
<h3>Islamorada, Florida</h3>
<h3>Nadi, Fiji</h3>
<h3>Waikoloa, Hawaii</h3>
<h3>Darwin, Australia</h3>
<h3>Key West, Florida</h3>
<h3>Bali, Indonesia</h3>
<h3>Durban, South Africa</h3>


In [3]:
import pandas as pd

In [4]:
dfdestinations = pd.read_csv('destinations.csv', delimiter=';')

In [5]:
dfdestinations

,Beachzone,Country,Latitude,Longitude
0,Ambergris Caye,Belize,17.952067,-87.970909
1,Bali,Indonesia,-8.330498,115.090640
2,Bermuda,Bermuda,32.301822,-64.760358
3,Bora Bora,French Polynesia,-16.504406,-151.736670
4,Darwin,Australia,-12.462288,130.840897
5,Durban,South Africa,-29.845409,31.030884
6,Islamorada,Florida,24.959866,-80.569984
7,Kauai,Hawaii,22.050467,-159.558768
8,Key West,Florida,24.585939,-81.688614
9,Ko Samui,Thailand,9.496833,99.993927


In [6]:
# create map of the world using latitude and longitude values
latitude = 0
longitude = 0
worldmap = folium.Map(location=[latitude, longitude], zoom_start=1)

# add markers to map
for latitude, longitude, country, Beachzone in zip(dfdestinations['Latitude'], 
                                                       dfdestinations['Longitude'], 
                                                       dfdestinations['Country'], 
                                                       dfdestinations['Beachzone']):
    label = '{}, {}'.format(country, Beachzone)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(worldmap)  
    
worldmap # with all tropical vacation destinations plotting as green dots

In [7]:
CLIENT_ID = 'DS5ZIBWFIDHV4TFIZLDHPGMZGK4TZ5DYDM5SUGXUM33IC1ZH' # your Foursquare ID
CLIENT_SECRET = 'VTPKT11F5FGPBDFSGBLIRDCYLCC1FHLZAGOFC0N0YNYWFPPL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: DS5ZIBWFIDHV4TFIZLDHPGMZGK4TZ5DYDM5SUGXUM33IC1ZH
CLIENT_SECRET:VTPKT11F5FGPBDFSGBLIRDCYLCC1FHLZAGOFC0N0YNYWFPPL


In [8]:
dfdestinations.loc[0,'Beachzone']

'Ambergris Caye'

In [9]:
dfdestinations_latitude = dfdestinations.loc[0, 'Latitude'] # neighborhood latitude value
dfdestinations_longitude = dfdestinations.loc[0, 'Longitude'] # neighborhood longitude value

dfdestinations_name = dfdestinations.loc[0, 'Beachzone'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(dfdestinations_name, 
                                                               dfdestinations_latitude, 
                                                              dfdestinations_longitude))

Latitude and longitude values of Ambergris Caye are 17.9520667, -87.97090920000001.


Lets use a radius of 10000 meters to assure values are found

In [10]:
LIMIT = 100
radius = 10000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    dfdestinations_latitude, 
    dfdestinations_longitude, 
    radius, 
    LIMIT,
    "university")
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DS5ZIBWFIDHV4TFIZLDHPGMZGK4TZ5DYDM5SUGXUM33IC1ZH&client_secret=VTPKT11F5FGPBDFSGBLIRDCYLCC1FHLZAGOFC0N0YNYWFPPL&v=20180605&ll=17.9520667,-87.97090920000001&radius=10000&limit=100'

In [11]:
#checking if it works and seeing the first results:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e261e9a006dce001bbd368f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 18.04206679000009,
    'lng': -87.87647982868437},
   'sw': {'lat': 17.86206660999991, 'lng': -88.06533857131565}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5702dee7cd10a02033e985e0',
       'name': 'Secret Beach',
       'location': {'address': 'Grand Belizean Estates',
        'lat': 17.973244523169704,
        'lng': -87.97200604223376,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
#creating a new dataframe of venues

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Secret Beach,Beach,17.973245,-87.972006
1,Rain Restaurant & Rooftop Lounge,Restaurant,17.945960,-87.944733
2,Estel's,Breakfast Spot,17.918856,-87.961876
3,Elvi's Kitchen,Seafood Restaurant,17.919799,-87.962351
4,The Truck Stop,Food Court,17.942350,-87.947798


In [14]:
#creating a function that finds nearby venues and loops ahead.
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,"4bf58dd8d48988d1ae941735")
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Beachzone', 
                  'Beachzone Latitude', 
                  'Beachzone Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
beachzone_venues = getNearbyVenues(names=dfdestinations['Beachzone'],
                                   latitudes=dfdestinations['Latitude'],
                                   longitudes=dfdestinations['Longitude']
                                  )

Ambergris Caye
Bali
Bermuda
Bora Bora
Darwin
Durban
Islamorada
Kauai
Key West
Ko Samui
Langkawi
Lombok
Nadi
Nassau
Port Douglas
St. Kitts and Nevis
Tamarindo
The Cook Islands
The Seychelles
Tulum
Waikoloa


In [16]:
print(beachzone_venues.shape)
beachzone_venues.head(10)

(1436, 7)


,Beachzone,Beachzone Latitude,Beachzone Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ambergris Caye,17.952067,-87.970909,Secret Beach,17.973245,-87.972006,Beach
1,Ambergris Caye,17.952067,-87.970909,Rain Restaurant & Rooftop Lounge,17.945960,-87.944733,Restaurant
2,Ambergris Caye,17.952067,-87.970909,Estel's,17.918856,-87.961876,Breakfast Spot
3,Ambergris Caye,17.952067,-87.970909,Elvi's Kitchen,17.919799,-87.962351,Seafood Restaurant
4,Ambergris Caye,17.952067,-87.970909,The Truck Stop,17.942350,-87.947798,Food Court
5,Ambergris Caye,17.952067,-87.970909,Palapa Bar & Grill,17.922736,-87.958394,Bar
6,Ambergris Caye,17.952067,-87.970909,Belize Chocolate Company,17.917579,-87.963101,Candy Store
7,Ambergris Caye,17.952067,-87.970909,Hurricane Bar,17.916411,-87.963777,Bar
8,Ambergris Caye,17.952067,-87.970909,Lily's Treasure Chest,17.920577,-87.960634,Caribbean Restaurant
9,Ambergris Caye,17.952067,-87.970909,Grand Caribe Resort,17.946478,-87.943803,Resort


In [17]:
#counting the venues per neighbourhood
beachzone_venues.groupby('Beachzone').count()

,Beachzone Latitude,Beachzone Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Beachzone,,,,,,
Ambergris Caye,71,71,71,71,71,71
Bali,25,25,25,25,25,25
Bermuda,68,68,68,68,68,68
Bora Bora,55,55,55,55,55,55
Darwin,85,85,85,85,85,85
Durban,100,100,100,100,100,100
Islamorada,100,100,100,100,100,100
Kauai,4,4,4,4,4,4
Key West,67,67,67,67,67,67


In [18]:
print('There are {} unique categories.'.format(len(beachzone_venues['Venue Category'].unique())))

There are 199 unique categories.


In [19]:
# one hot encoding
beach_onehot = pd.get_dummies(beachzone_venues[['Venue Category']], prefix="", prefix_sep="")


In [20]:
beachzone_venues.head()

,Beachzone,Beachzone Latitude,Beachzone Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ambergris Caye,17.952067,-87.970909,Secret Beach,17.973245,-87.972006,Beach
1,Ambergris Caye,17.952067,-87.970909,Rain Restaurant & Rooftop Lounge,17.945960,-87.944733,Restaurant
2,Ambergris Caye,17.952067,-87.970909,Estel's,17.918856,-87.961876,Breakfast Spot
3,Ambergris Caye,17.952067,-87.970909,Elvi's Kitchen,17.919799,-87.962351,Seafood Restaurant
4,Ambergris Caye,17.952067,-87.970909,The Truck Stop,17.942350,-87.947798,Food Court


In [21]:
beach_onehot.head()

,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cave,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Donut Shop,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Other Great Outdoors,Paper / Office Supplies Store,Park,Pharmacy,Pier,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Racetrack,Rafting,Resort,Restaurant,River,Russian Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Som Tum Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Wine Bar,Yoga Studio,Yucatecan Restaurant,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
# one hot encoding
beach_onehot = pd.get_dummies(beachzone_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
beach_onehot['Beachzone'] = beachzone_venues['Beachzone']  

neigh = beach_onehot['Beachzone']
beach_onehot.drop(labels=['Beachzone'], axis=1,inplace = True)
beach_onehot.insert(0, 'Beachzone', neigh)
beach_onehot.head()


,Beachzone,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cave,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Donut Shop,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Other Great Outdoors,Paper / Office Supplies Store,Park,Pharmacy,Pier,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Racetrack,Rafting,Resort,Restaurant,River,Russian Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Som Tum Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Wine Bar,Yoga Studio,Yucatecan Restaurant,Zoo
0,Ambergris Caye,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ambergris Caye,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ambergris Caye,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ambergris Caye,0,0,0,0,0,0

In [23]:
beach_onehot.shape

(1436, 200)

In [24]:
beach_grouped = beach_onehot.groupby('Beachzone').mean().reset_index()
beach_grouped.head()

,Beachzone,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cave,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Donut Shop,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Other Great Outdoors,Paper / Office Supplies Store,Park,Pharmacy,Pier,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Racetrack,Rafting,Resort,Restaurant,River,Russian Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Som Tum Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Wine Bar,Yoga Studio,Yucatecan Restaurant,Zoo
0,Ambergris Caye,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.014085,0.00,0.140845,0.000000,0.056338,0.014085,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.014085,0.000000,0.0,0.0,0.0,0.0,0.0,0.014085,0.0,0.014085,0.140845,0.000000,0.0,0.0,0.0,0.014085,0.028169,0.0,0.0,0.0,0.014085,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.014085,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.014085,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.028169,0.000000,0.014085,0.0,0.014085,0.000000,0.000000,0.00,0.0,0.014085,0.000000,0.000000,0.0,0.000000,0.028169,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.014085,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.014085,0.0,0.00,0.000000,0.014085,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.197183,0.056338,0.0,0.0,0.0,0.000000,0.014085,0.0,0.028169,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.014085,0.0,0.0,0.000000,0.0,0.014085,0.000000,0.0,0.000000,0.014085,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.014085,0.0

In [25]:
num_top_venues = 5

for beach in beach_grouped['Beachzone']:
    print("----"+beach+"----")
    temp = beach_grouped[beach_grouped['Beachzone'] == beach].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ambergris Caye----
                  venue  freq
0                Resort  0.20
1                   Bar  0.14
2  Caribbean Restaurant  0.14
3                 Beach  0.06
4            Restaurant  0.06


----Bali----
                   venue  freq
0             Restaurant  0.16
1       Asian Restaurant  0.12
2                  Hotel  0.12
3  Indonesian Restaurant  0.12
4    Balinese Restaurant  0.08


----Bermuda----
        venue  freq
0  Restaurant  0.15
1       Beach  0.12
2       Hotel  0.06
3      Resort  0.06
4        Café  0.04


----Bora Bora----
               venue  freq
0  French Restaurant  0.18
1             Resort  0.18
2         Restaurant  0.09
3          Hotel Bar  0.07
4              Beach  0.05


----Darwin----
                   venue  freq
0                   Café  0.09
1  Australian Restaurant  0.06
2                    Pub  0.05
3                  Hotel  0.05
4       Asian Restaurant  0.04


----Durban----
                venue  freq
0                Café  0.12


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
# Create a dataframe with top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Beachzone']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
beach_venues_sorted = pd.DataFrame(columns=columns)
beach_venues_sorted['Beachzone'] = beach_grouped['Beachzone']

for ind in np.arange(beach_grouped.shape[0]):
    beach_venues_sorted.iloc[ind, 1:] = return_most_common_venues(beach_grouped.iloc[ind, :], num_top_venues)

beach_venues_sorted.head()

,Beachzone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ambergris Caye,Resort,Caribbean Restaurant,Bar,Restaurant,Beach,Seafood Restaurant,Latin American Restaurant,Coffee Shop,Hotel,Beach Bar
1,Bali,Restaurant,Indonesian Restaurant,Hotel,Asian Restaurant,Balinese Restaurant,Farmers Market,Motel,Scenic Lookout,Mountain,Temple
2,Bermuda,Restaurant,Beach,Hotel,Resort,Café,Sushi Restaurant,Bar,Seafood Restaurant,Sports Bar,Park
3,Bora Bora,Resort,French Restaurant,Restaurant,Hotel Bar,Beach,Hotel Pool,Seafood Restaurant,Lake,Sushi Restaurant,Hotel
4,Darwin,Café,Australian Restaurant,Pub,Hotel,Beach,Fast Food Restaurant,Asian Restaurant,Farmers Market,Park,Japanese Restaurant


In [28]:
# Run K-means to break up into clusters
kclusters = 10

beach_grouped_clustering = beach_grouped.drop('Beachzone', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(beach_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([9, 6, 8, 5, 0, 0, 1, 4, 3, 8, 1, 2, 3, 8, 0, 3, 3, 7, 8, 3, 8],
      dtype=int32)

In [29]:
beach_venues_sorted.columns

Index(['Beachzone', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [30]:
# Create dataframe that includes the cluster and top 10 venues

# Add clustering labels
beach_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

beach_merged = dfdestinations

# Merge toronto_venues_sorted with dftoronto to add latitude/longitude for each neighborhood
beach_merged = beach_merged.join(beach_venues_sorted.set_index('Beachzone'), on='Beachzone')

beach_merged.tail(10) # check the last columns!

,Beachzone,Country,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Lombok,Indonesia,-8.586667,116.337646,2,Asian Restaurant,Hotel,Arcade,Other Great Outdoors,Diner,Lake,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop
12,Nadi,Fiji,-17.799339,177.417206,3,Hotel,Seafood Restaurant,Restaurant,Resort,Café,Bar,Breakfast Spot,Indian Restaurant,Convenience Store,Mediterranean Restaurant
13,Nassau,Bahamas,25.076799,-77.338295,8,Resort,Beach,Caribbean Restaurant,Bar,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Plaza
14,Port Douglas,Australia,-16.483978,145.465836,0,Australian Restaurant,Café,Hotel,Resort,Coffee Shop,Thai Restaurant,Ice Cream Shop,Seafood Restaurant,Boat or Ferry,Italian Restaurant
15,St. Kitts and Nevis,The Caribbean,17.315342,-62.744354,3,Hotel,Seafood Restaurant,Bar,Caribbean Restaurant,Café,Casino,Bus Stop,Beach,Mexican Restaurant,Soccer Stadium
16,Tamarindo,Costa Rica,10.298537,-85.839813,3,Hotel,Beach,Resort,Bed & Breakfast,Surf Spot,Bar,Italian Restaurant,Seafood Restaurant,Pizza Place,Hostel
17,The Cook Islands,South Pacific,-21.235806,-159.778488,7,Café,Restaurant,Theater,Snack Place,Resort,Seafood Restaurant,Surf Spot,Sandwich Place,Miscellaneous Shop,Hostel
18,The Seychelles,East Africa,-4.677849,55.467182,8,Resort,Beach,Café,Seafood Restaurant,Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Cocktail Bar,African Restaurant,Historic Site
19,Tulum,Mexico,20.211163,-87.462784,3,Hotel,Beach,Mexican Restaurant,Seafood Restaurant,Taco Place,Ice Cream Shop,Café,Italian Restaurant,Caribbean Restaurant,Historic Site
20,Waikoloa,Hawaii,19.942563,-155.786490,8,Beach,Resort,Golf Course,Seafood Restaurant,Hotel,American Restaurant,Grocery Store,Restaurant,Breakfast Spot,Sandwich Place


In [31]:
# Visualize the clustered data
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=1)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(beach_merged['Latitude'], beach_merged['Longitude'], beach_merged['Beachzone'], beach_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [32]:
beach_merged.loc[beach_merged['Cluster Labels'] == 0, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Australia,Café,Australian Restaurant,Pub,Hotel,Beach,Fast Food Restaurant,Asian Restaurant,Farmers Market,Park,Japanese Restaurant
5,South Africa,Café,Coffee Shop,Burger Joint,Indian Restaurant,Restaurant,Italian Restaurant,Pub,Hotel,Stadium,Bakery
14,Australia,Australian Restaurant,Café,Hotel,Resort,Coffee Shop,Thai Restaurant,Ice Cream Shop,Seafood Restaurant,Boat or Ferry,Italian Restaurant


In [33]:
beach_merged.loc[beach_merged['Cluster Labels'] == 1, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Florida,Resort,Seafood Restaurant,Beach,Bar,Gas Station,Bakery,Breakfast Spot,Café,Brewery,Sandwich Place
10,Malaysia,Malay Restaurant,Resort,Asian Restaurant,Hotel,Thai Restaurant,Seafood Restaurant,Beach,Candy Store,Café,Restaurant


In [34]:
beach_merged.loc[beach_merged['Cluster Labels'] == 2, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Indonesia,Asian Restaurant,Hotel,Arcade,Other Great Outdoors,Diner,Lake,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop


In [35]:
beach_merged.loc[beach_merged['Cluster Labels'] == 3, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Florida,Hotel,Harbor / Marina,Grocery Store,Cuban Restaurant,Pizza Place,Bar,Pharmacy,Resort,Beach,Seafood Restaurant
12,Fiji,Hotel,Seafood Restaurant,Restaurant,Resort,Café,Bar,Breakfast Spot,Indian Restaurant,Convenience Store,Mediterranean Restaurant
15,The Caribbean,Hotel,Seafood Restaurant,Bar,Caribbean Restaurant,Café,Casino,Bus Stop,Beach,Mexican Restaurant,Soccer Stadium
16,Costa Rica,Hotel,Beach,Resort,Bed & Breakfast,Surf Spot,Bar,Italian Restaurant,Seafood Restaurant,Pizza Place,Hostel
19,Mexico,Hotel,Beach,Mexican Restaurant,Seafood Restaurant,Taco Place,Ice Cream Shop,Café,Italian Restaurant,Caribbean Restaurant,Historic Site


In [36]:
beach_merged.loc[beach_merged['Cluster Labels'] == 4, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Hawaii,Scenic Lookout,Asian Restaurant,Mountain,Dive Spot,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit


In [37]:
beach_merged.loc[beach_merged['Cluster Labels'] == 5, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,French Polynesia,Resort,French Restaurant,Restaurant,Hotel Bar,Beach,Hotel Pool,Seafood Restaurant,Lake,Sushi Restaurant,Hotel


In [38]:
beach_merged.loc[beach_merged['Cluster Labels'] == 6, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Indonesia,Restaurant,Indonesian Restaurant,Hotel,Asian Restaurant,Balinese Restaurant,Farmers Market,Motel,Scenic Lookout,Mountain,Temple


In [39]:
beach_merged.loc[beach_merged['Cluster Labels'] == 7, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,South Pacific,Café,Restaurant,Theater,Snack Place,Resort,Seafood Restaurant,Surf Spot,Sandwich Place,Miscellaneous Shop,Hostel


In [40]:
beach_merged.loc[beach_merged['Cluster Labels'] == 8, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bermuda,Restaurant,Beach,Hotel,Resort,Café,Sushi Restaurant,Bar,Seafood Restaurant,Sports Bar,Park
9,Thailand,Resort,Thai Restaurant,Hotel,Beach,Cocktail Bar,Italian Restaurant,Café,Scenic Lookout,Restaurant,Indian Restaurant
13,Bahamas,Resort,Beach,Caribbean Restaurant,Bar,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Plaza
18,East Africa,Resort,Beach,Café,Seafood Restaurant,Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Cocktail Bar,African Restaurant,Historic Site
20,Hawaii,Beach,Resort,Golf Course,Seafood Restaurant,Hotel,American Restaurant,Grocery Store,Restaurant,Breakfast Spot,Sandwich Place


In [41]:
beach_merged.loc[beach_merged['Cluster Labels'] == 9, beach_merged.columns[[1] + list(range(5, beach_merged.shape[1]))                                                
                                                                                ]]

,Country,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belize,Resort,Caribbean Restaurant,Bar,Restaurant,Beach,Seafood Restaurant,Latin American Restaurant,Coffee Shop,Hotel,Beach Bar
